In [1]:
import sympy
import underworld3 as uw
import numpy as np
import os 

os.environ["UW_TIMING_ENABLE"] = "1"
os.environ["UW_JITNAME"] = "TESTF"

# These are tested by test_001_meshes.py

mesh = uw.meshing.UnstructuredSimplexBox(
    cellSize=0.1, regular=True, qdegree=2, refinement=0
)

if mesh.dim == 2:
    x, y = mesh.X
else:
    x, y, z = mesh.X

u = uw.discretisation.MeshVariable("U", mesh, mesh.dim, 
                                   vtype=uw.VarType.VECTOR, 
                                   degree=2, varsymbol=r"\mathbf{u}",
)
p = uw.discretisation.MeshVariable("P",
    mesh, 1, vtype=uw.VarType.SCALAR, degree=1, continuous=True, 
                                   varsymbol=r"\mathbf{p}", 
)

stokes = uw.systems.Stokes(mesh, velocityField=u, pressureField=p, verbose=True)
stokes.constitutive_model = (
    uw.systems.constitutive_models.ViscoElasticPlasticFlowModel(u)
)
stokes.constitutive_model.Parameters.shear_viscosity_0 = 1

stokes.petsc_options["snes_type"] = "newtonls"
stokes.petsc_options["ksp_type"] = "fgmres"
stokes.petsc_options["ksp_monitor"] = None
stokes.petsc_options["snes_monitor"] = None
stokes.tolerance = 1.0e-3

# stokes.petsc_options.setValue("fieldsplit_velocity_pc_type", "mg")
stokes.petsc_options.setValue("fieldsplit_velocity_pc_mg_type", "kaskade")
stokes.petsc_options.setValue("fieldsplit_velocity_pc_mg_cycle_type", "w")

stokes.petsc_options["fieldsplit_velocity_mg_coarse_pc_type"] = "svd"
stokes.petsc_options[f"fieldsplit_velocity_ksp_type"] = "fcg"
stokes.petsc_options[f"fieldsplit_velocity_mg_levels_ksp_type"] = "chebyshev"
stokes.petsc_options[f"fieldsplit_velocity_mg_levels_ksp_max_it"] = 7
stokes.petsc_options[f"fieldsplit_velocity_mg_levels_ksp_converged_maxits"] = None

stokes.petsc_options.setValue("fieldsplit_pressure_pc_type", "gamg")
stokes.petsc_options.setValue("fieldsplit_pressure_pc_mg_type", "multiplicative")
stokes.petsc_options.setValue("fieldsplit_pressure_pc_mg_cycle_type", "v")

if mesh.dim == 2:
    stokes.bodyforce = sympy.Matrix([0.0, 1.0*sympy.sin(x)])

    stokes.add_natural_bc((0.0, -u[1].sym), "Top")
    # stokes.add_dirichlet_bc((sympy.oo, 0.0), "Top")
    stokes.add_dirichlet_bc((sympy.oo,0.0), "Bottom")
    stokes.add_dirichlet_bc((0.0,sympy.oo), "Left")
    stokes.add_dirichlet_bc((0.0,sympy.oo), "Right")

else:
    stokes.bodyforce = sympy.Matrix([0, sympy.sin(x), 0])

    stokes.add_essential_bc([0.0, 0.0, 0.0], "Top")
    stokes.add_essential_bc([0.0, 0.0, 0.0], "Bottom")
    stokes.add_essential_bc([0.0, 0.0, 0.0], "Left")
    stokes.add_essential_bc([0.0, 0.0, 0.0], "Right")    
    stokes.add_essential_bc([0.0, 0.0, 0.0], "Front")
    stokes.add_essential_bc([0.0, 0.0, 0.0], "Back")


In [2]:
with uw.utilities.CaptureStdout(split=False) as captured_dm_view:
    mesh.dm.view()


In [3]:
stokes.solve(verbose=False, debug=False)
stokes.dm.ds.view()

## assert stokes.snes.getConvergedReason() > 0

Stokes: Jacobians complete, now compile
Compiler: 
 0: Matrix([[0, -1.0*sin(N.x)]])
 1: Matrix([[-\mathbf{p}(N.x, N.y) + 2*\mathbf{u}_{ 0,0}(N.x, N.y), \mathbf{u}_{ 0,1}(N.x, N.y) + \mathbf{u}_{ 1,0}(N.x, N.y)], [\mathbf{u}_{ 0,1}(N.x, N.y) + \mathbf{u}_{ 1,0}(N.x, N.y), -\mathbf{p}(N.x, N.y) + 2*\mathbf{u}_{ 1,1}(N.x, N.y)]])
 2: Matrix([[\mathbf{u}_{ 0,0}(N.x, N.y) + \mathbf{u}_{ 1,1}(N.x, N.y)]])
 3: Matrix([[oo], [0]])
 4: Matrix([[0], [oo]])
 5: Matrix([[0], [oo]])
 6: Matrix([[0, 0], [0, 0]])
 7: Matrix([[0, 0, 0, 0], [0, 0, 0, 0]])
 8: Matrix([[0, 0], [0, 0], [0, 0], [0, 0]])
 9: Matrix([[2, 0, 0, 1], [0, 0, 1, 0], [0, 1, 0, 0], [1, 0, 0, 2]])
 10: Matrix([[0], [0]])
 11: Matrix([[0, 0], [0, 0]])
 12: Matrix([[-1, 0], [0, -1]])
 13: Matrix([[0, 0], [0, 0], [0, 0], [0, 0]])
 14: Matrix([[0], [0]])
 15: Matrix([[1], [0], [0], [1]])
 16: 1
 17: Matrix([[0], [-\mathbf{u}_{ 1 }(N.x, N.y)]])
 18: Matrix([[0, 0], [0, -1]])
 19: Matrix([[0, 0, 0, 0], [0, 0, 0, 0]])
Setting bc 0 (natural

In [4]:
for pen in [10, 33, 100, 330, 1000]:
    stokes.natural_bcs = []
    stokes.dm = None
    stokes.add_natural_bc((0.0, pen*u[1].sym), "Top")

    
    stokes.solve(zero_init_guess=False, _force_setup=True)

    

Stokes: Jacobians complete, now compile
Compiler: 
 0: Matrix([[0, -1.0*sin(N.x)]])
 1: Matrix([[-\mathbf{p}(N.x, N.y) + 2*\mathbf{u}_{ 0,0}(N.x, N.y), \mathbf{u}_{ 0,1}(N.x, N.y) + \mathbf{u}_{ 1,0}(N.x, N.y)], [\mathbf{u}_{ 0,1}(N.x, N.y) + \mathbf{u}_{ 1,0}(N.x, N.y), -\mathbf{p}(N.x, N.y) + 2*\mathbf{u}_{ 1,1}(N.x, N.y)]])
 2: Matrix([[\mathbf{u}_{ 0,0}(N.x, N.y) + \mathbf{u}_{ 1,1}(N.x, N.y)]])
 3: Matrix([[oo], [0]])
 4: Matrix([[0], [oo]])
 5: Matrix([[0], [oo]])
 6: Matrix([[0, 0], [0, 0]])
 7: Matrix([[0, 0, 0, 0], [0, 0, 0, 0]])
 8: Matrix([[0, 0], [0, 0], [0, 0], [0, 0]])
 9: Matrix([[2, 0, 0, 1], [0, 0, 1, 0], [0, 1, 0, 0], [1, 0, 0, 2]])
 10: Matrix([[0], [0]])
 11: Matrix([[0, 0], [0, 0]])
 12: Matrix([[-1, 0], [0, -1]])
 13: Matrix([[0, 0], [0, 0], [0, 0], [0, 0]])
 14: Matrix([[0], [0]])
 15: Matrix([[1], [0], [0], [1]])
 16: 1
 17: Matrix([[0], [10*\mathbf{u}_{ 1 }(N.x, N.y)]])
 18: Matrix([[0, 0], [0, 10]])
 19: Matrix([[0, 0, 0, 0], [0, 0, 0, 0]])
Setting bc 0 (natur

In [5]:
import mpi4py

if uw.mpi.size == 1:

    import numpy as np
    import pyvista as pv
    import vtk

    pv.global_theme.background = "white"
    pv.global_theme.window_size = [750, 1200]
    pv.global_theme.anti_aliasing = "msaa"
    pv.global_theme.jupyter_backend = "panel"
    pv.global_theme.smooth_shading = True

    mesh.vtk("tmp_mesh.vtk")
    pvmesh = pv.read("tmp_mesh.vtk")

    V = u

    pvmesh.point_data["P"] = uw.function.evalf(stokes.p.sym[0], mesh.data)
    pvmesh.point_data["V"] = uw.function.evalf(V.sym.dot(V.sym), mesh.data)

    arrow_loc = np.zeros((V.coords.shape[0], 3))
    arrow_loc[:, 0:2] = V.coords[...]

    arrow_length = np.zeros((V.coords.shape[0], 3))
    arrow_length[:, 0] = uw.function.evalf(V.sym[0], V.coords)
    arrow_length[:, 1] = uw.function.evalf(V.sym[1], V.coords)

    pl = pv.Plotter(window_size=[1000, 1000])
    pl.add_axes()

    pl.add_mesh(
        pvmesh,
        cmap="coolwarm",
        edge_color="Black",
        show_edges=True,
        scalars="P",
        use_transparency=False,
        opacity=1.0,
    )

    # pl.add_mesh(pvmesh, cmap="coolwarm", edge_color="Black", show_edges=True, scalars="T",
    #               use_transparency=False, opacity=1.0)

    pl.add_arrows(arrow_loc, arrow_length, mag=10)

    pl.show(cpos="xy")

In [6]:
arrow_length[:,0].min()

-0.01156323794184607

In [9]:
stokes.natural_bcs[0].fns["f0"] = "test"

'test'